In [4]:
#librerias Keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.layers import LSTM, GRU
import matplotlib.pyplot as plt

In [6]:
from utils.multi_gpu import make_parallel
from dataset.dataset import leer
from keras.utils.np_utils import to_categorical

## 1 CNNs en CIFAR10
---

En esta seccion, experimentaremos con redes convolucionales, conocidas como CNNs o ConvNets. Para ello y con el fin de comparar con los resultados obtenidos por redes FF, reutilizaremos el dataset CIFAR10 utilizado en la tarea anterior. Si no dispone ya del dataset en su maquina, por favor siga a las instrucciones de descarga entregadas en la tarea anterior. **Nota: Para esta actividad es bastante aconsejable entrenar las redes usando una GPU, ya que de otro modo los tiempos de entrenamiento son largos[1].**

En la imagen que vemos a continuación podemos hacernos una idea de cómo funciona una una red convolucional, con capas convolucionales, capas de pooling y capas full conectadas finales(post Flaten):


![alt text](convnet.jpg  "Title")

**[1]** Respecto al uso de GPU, para las experiencia se fue mas allá de los solicitado y se preparo un ambiente de ejecución paralelo en múltiples GPU, para mas detalles de este proceso ver Anexo 1-Procesamiento Paralelo. Podemos ver todas las GPU’s disponibles con el comando de Nvidia:


In [ ]:
!nvidia-smi

Fri Jun 16 00:41:15 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.39                 Driver Version: 375.39                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:17.0     Off |                    0 |
| N/A   62C    P0    60W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 0000:00:18.0     Off |                    0 |
| N/A   50C    P0    71W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

Objetivo
---


**(a)** Prepare subconjuntos de entrenamiento, validacion y pruebas identicos a los que utilizo en el ultimo ıtem de la tarea 1. Normalice las imagenes de entrenamiento y pruebas, dividiendo las intensidades originales de pixel en cada canal por 255. Es importante recordar que en la tarea1 representamos las imagenes como un vector unidimensional. Por lo tanto, antes de trabajar con CNNs sera necesario recuperar la forma original de las imagenes. Ademas, si desea trabajar con el orden de las dimensiones denominado ’tf’ (por defecto para TensorFlow†) debera hacer realizar la transposicion correspondiente.
Finalmente, genere una representacion adecuada de las salidas deseadas de la red.

In [9]:
%%time
URL='data/'
Xtr,Ytr,Xt,Yt,Xv,Yv = leer(URL)
#Datos de entrenamiento
X_train = Xtr.reshape((Xtr.shape[0],32,32,3))
X_train = X_train.transpose([0, 3, 2, 1]) 
Y_train = to_categorical(Ytr, 10)
#Datos de Test
X_test  = Xt.reshape((Xt.shape[0],32,32,3))
X_test  = X_test.transpose([0, 3, 2, 1])
Y_test  = to_categorical(Yt, 10)
#Datos de Validación
X_val   = Xv.reshape((Xv.shape[0],32,32,3))
X_val   = X_val.transpose([0, 3, 2, 1])
Y_val   = to_categorical(Yv, 10)

X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_val   = X_val.astype('float32')

CPU times: user 1.5 s, sys: 859 ms, total: 2.36 s
Wall time: 2.47 s


In [10]:
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)
(50000, 10)
(10000, 10)


In [11]:
_size, n_channels, n_rows, n_cols = X_train.shape
n_classes = 10


**(b)** Defina una CNN con arquitectura **C × P × C × P × F × F** . Para ambas capas convolucionales utilice 64 filtros de 3 × 3 y funciones de activaci ́on ReLu. Para las capas de pooling utilice filtros de 2 × 2 con stride 2. Para la capa MLP escondida use 512 neuronas. Genere un esquema lo mas compacto posible que muestre los cambios de forma (dimensionalidad) que experimenta un patron de entrada a medida que se ejecuta un forward-pass y el numero de parametros de cada capa.

In [12]:
model = Sequential()
model.add(Convolution2D(64, (3, 3), padding='same', input_shape=(n_channels, n_rows, n_cols)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 32, 32)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 16, 16)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 16, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 8, 8)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________